In [1]:
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# record time before execution
start_time = time.time()
# execute operation
result = 5 + 2
# record time after execution
end_time = time.time()
print("Result calculated in {} sec".format(end_time - start_time))

Result calculated in 0.0 sec


In [3]:
#using List comprehension 

list_comp_start_time = time.time()
result = [i*i for i in range(0,1000000)]
list_comp_end_time = time.time()
print("Time using the list_comprehension: {} sec".format(list_comp_end_time -
list_comp_start_time))

Time using the list_comprehension: 0.07301616668701172 sec


In [4]:
# Using For loop

for_loop_start_time= time.time()
result=[]
for i in range(0,1000000):
    result.append(i*i)
for_loop_end_time= time.time()
print("Time using the for loop: {} sec".format(for_loop_end_time - for_loop_start_time))

Time using the for loop: 0.15406060218811035 sec


In [5]:
list_comp_time = list_comp_end_time - list_comp_start_time
for_loop_time = for_loop_end_time - for_loop_start_time
print("Difference in time: {} %".format((for_loop_time - list_comp_time)/
list_comp_time*100))

Difference in time: 110.99519350077713 %


In [6]:
def sum_brute_force(N):
    res = 0
    for i in range(1,N+1):
        res+=i
    return res

# Using brute force
bf_start_time = time.time()
bf_result = sum_brute_force(1000000)
bf_end_time = time.time()

print("Time using brute force: {} sec".format(bf_end_time - bf_start_time))

Time using brute force: 0.043009042739868164 sec


In [7]:
def sum_formula(N):
    return N*(N+1)/2
  
# Using the formula
formula_start_time = time.time()
formula_result = sum_formula(1000000)
formula_end_time = time.time()

print("Time using the formula: {} sec".format(formula_end_time - formula_start_time))

Time using the formula: 0.0 sec


In [8]:
restaurant = pd.read_csv('data/restaurant_data.csv')

restaurant_grouped = restaurant.groupby('smoker')
print(restaurant_grouped.count())

        total_bill  tip  sex  day  time  size
smoker                                       
No             151  151  151  151   151   151
Yes             93   93   93   93    93    93


In [9]:
zscore = lambda x: (x - x.mean() ) / x.std()

restaurant_grouped = restaurant.groupby('time')

restaurant_transformed = restaurant_grouped.transform(zscore)
restaurant_transformed.head()

,total_bill,tip,size
0,-0.416446,-1.457045,-0.692873
1,-1.143855,-1.004475,0.405737
2,0.023282,0.276645,0.405737
3,0.315339,0.144355,-0.692873
4,0.414880,0.353234,1.504347


In [10]:
restaurant.groupby('sex').transform(zscore)

mean_female = restaurant.groupby('sex').mean()['total_bill']['Female']
mean_male = restaurant.groupby('sex').mean()['total_bill']['Male']
std_female = restaurant.groupby('sex').std()['total_bill']['Female']
std_male = restaurant.groupby('sex').std()['total_bill']['Male']

for i in range(len(restaurant)):
    if restaurant.iloc[i][2] == 'Female':
        restaurant.iloc[i][0] = (restaurant.iloc[i][0] - mean_female)/std_female
    else:
        restaurant.iloc[i][0] = (restaurant.iloc[i][0] - mean_male)/std_male

In [11]:
prior_counts = restaurant.groupby('time')
prior_counts['total_bill'].count()

time
Dinner    176
Lunch      68
Name: total_bill, dtype: int64

In [12]:
p = 0.1 #percentage missing data required

mask = np.random.choice([np.nan,1], size=len(restaurant), p=[p,1-p])
restaurant_nan = restaurant.copy()
restaurant_nan['total_bill'] =  restaurant_nan['total_bill'] * mask

In [13]:
prior_counts = restaurant.groupby('time')
prior_counts['total_bill'].count()

time
Dinner    176
Lunch      68
Name: total_bill, dtype: int64

In [14]:
# Missing value imputation

missing_trans = lambda x: x.fillna(x.mean())
restaurant_nan_grouped = restaurant_nan.groupby('time')['total_bill']
restaurant_nan_grouped.transform(missing_trans)

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [15]:
start_time = time.time()
mean_din = restaurant_nan.loc[restaurant_nan.time =='Dinner']['total_bill'].mean()
mean_lun = restaurant_nan.loc[restaurant_nan.time == 'Lunch']['total_bill'].mean()

for row in range(len(restaurant_nan)):
    if restaurant_nan.iloc[row]['time'] == 'Dinner':
        restaurant_nan.loc[row, 'total_time'] = mean_din
    else:
        restaurant_nan.loc[row, 'total_time'] = mean_lun
print("Results from the above operation calculated in %s seconds" % (time.time() - start_time))

Results from the above operation calculated in 0.0600123405456543 seconds


In [16]:
restaurant_grouped = restaurant.groupby('day')
filter_trans = lambda x : x['total_bill'].mean() > 20
restaurant_filtered = restaurant_grouped.filter(filter_trans)

In [17]:
print(restaurant_filtered['tip'].mean())

3.1152760736196328


In [18]:
print(restaurant['tip'].mean())

2.9982786885245902


In [19]:
t=[restaurant.loc[restaurant['day'] == i]['tip'] for i in restaurant['day'].unique()
  if restaurant.loc[restaurant['day'] == i]['total_bill'].mean()>20]
restaurant_filtered = t[0]

for j in t[1:]:
    restaurant_filtered=restaurant_filtered.append(j,ignore_index=True)

In [20]:
restaurant_filtered

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
158    4.67
159    5.92
160    2.00
161    2.00
162    1.75
Name: tip, Length: 163, dtype: float64